# Implementing Energy Function

In [1]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from math import tanh
from pathlib import Path
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt

from idpconfgen.libs.libstructure import Structure, col_element, col_name, col_resName, col_resSeq
from idpconfgen.core.build_definitions import (
    generate_residue_template_topology, read_ff14SB_params,
    expand_topology_bonds_apart, topology_3_bonds_apart,
    inter_res_exact_3_bonds,
)

%matplotlib inline

## Preparating Structures and Parameters

### Loading structures

Here we will load two conformers, one clash-free and other with clashes, that were created by the current version of `IDPConfGen` where clashes were computed with a simple hard sphere clash optimization, which we want to optimize here via an energy function. We will use `Structure()` to extract the coordinates and labels of the conformers. Keep in mind that, when during the building process, coordinates and labels will be already in memory.

In [2]:
good = Path('conformer_good_amber.pdb')
bad = Path('conformer_bad_amber.pdb')

In [3]:
# good
gs = Structure(good)
gs.build()
gcoods = gs.coords

# bad
bs = Structure(bad)
bs.build()
bcoods = bs.coords

# atoms labels are the same for both conformers
elements = gs.data_array[:, col_element]
print(set(elements))

# atom names
atom_names = gs.data_array[:, col_name]
print(atom_names)

# res type
res_type = gs.data_array[:, col_resName]
dict.fromkeys(res_type).keys()

# res number
res_num = list(dict.fromkeys(gs.data_array[:, col_resSeq]).keys())

print(res_num)

{'C', 'O', 'N', 'H'}
['N' 'CA' 'C' 'O' 'CB' 'HA' 'HB1' 'HB2' 'HB3' 'N' 'CA' 'C' 'O' 'CB' 'H'
 'HA' 'HB1' 'HB2' 'HB3' 'N' 'CA' 'C' 'O' 'CB' 'H' 'HA' 'HB1' 'HB2' 'HB3'
 'N' 'CA' 'C' 'O' 'CB' 'H' 'HA' 'HB1' 'HB2' 'HB3' 'N' 'CA' 'C' 'O' 'CB'
 'H' 'HA' 'HB1' 'HB2' 'HB3' 'N' 'CA' 'C' 'O' 'CB' 'H' 'HA' 'HB1' 'HB2'
 'HB3' 'N' 'CA' 'C' 'O' 'CB' 'H' 'HA' 'HB1' 'HB2' 'HB3' 'N' 'CA' 'C' 'O'
 'CB' 'H' 'HA' 'HB1' 'HB2' 'HB3' 'N' 'CA' 'C' 'O' 'CB' 'H' 'HA' 'HB1'
 'HB2' 'HB3' 'N' 'CA' 'C' 'O' 'CB' 'H' 'HA' 'HB1' 'HB2' 'HB3' 'N' 'CA' 'C'
 'O' 'CB' 'H' 'HA' 'HB1' 'HB2' 'HB3' 'OXT']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']


### Loading parameters from Force Fields

This is a crucial part as parameters need to be in tune with each other.

I will build the parameters here as I implement the different energy functions.

In [4]:
ff14SB = read_ff14SB_params()

In [5]:
pprint(ff14SB.keys())

dict_keys(['protein-C', 'protein-CA', 'protein-CB', 'protein-CC', 'protein-CN', 'protein-CR', 'protein-CT', 'protein-CV', 'protein-CW', 'protein-C*', 'protein-CX', 'protein-H', 'protein-HC', 'protein-H1', 'protein-HA', 'protein-H4', 'protein-H5', 'protein-HO', 'protein-HS', 'protein-HP', 'protein-N', 'protein-NA', 'protein-NB', 'protein-N2', 'protein-N3', 'protein-O', 'protein-O2', 'protein-OH', 'protein-S', 'protein-SH', 'protein-CO', 'protein-2C', 'protein-3C', 'protein-C8', 'ALA', 'ARG', 'ASH', 'ASN', 'ASP', 'CYM', 'CYS', 'CYX', 'GLH', 'GLN', 'GLU', 'GLY', 'HID', 'HIE', 'HIP', 'HYP', 'ILE', 'LEU', 'LYN', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'CALA', 'CARG', 'CASN', 'CASP', 'CCYS', 'CCYX', 'CGLN', 'CGLU', 'CGLY', 'CHID', 'CHIE', 'CHIP', 'CHYP', 'CILE', 'CLEU', 'CLYS', 'CMET', 'CPHE', 'CPRO', 'CSER', 'CTHR', 'CTRP', 'CTYR', 'CVAL', 'NHE', 'NME', 'ACE', 'NALA', 'NARG', 'NASN', 'NASP', 'NCYS', 'NCYX', 'NGLN', 'NGLU', 'NGLY', 'NHID', 'NHIE', 'NHIP', 'NILE', 'NLEU

In [6]:
ff14SB['ALA']

{'N': {'charge': '-0.4157', 'type': 'protein-N'},
 'H': {'charge': '0.2719', 'type': 'protein-H'},
 'CA': {'charge': '0.0337', 'type': 'protein-CX'},
 'HA': {'charge': '0.0823', 'type': 'protein-H1'},
 'CB': {'charge': '-0.1825', 'type': 'protein-CT'},
 'HB1': {'charge': '0.0603', 'type': 'protein-HC'},
 'HB2': {'charge': '0.0603', 'type': 'protein-HC'},
 'HB3': {'charge': '0.0603', 'type': 'protein-HC'},
 'C': {'charge': '0.5973', 'type': 'protein-C'},
 'O': {'charge': '-0.5679', 'type': 'protein-O'}}

### Prepares data arrays

Prepares data before performing calculations, in order to allow for parameters indexing.

## Defining Energy Terms

... and implementation

### vdW term

$$
E_{vdW} = 
\begin{cases}
      \infty, \qquad\qquad\qquad\qquad\qquad\quad r_{ij} < r_{ij}^{*} \\
      4\varepsilon_{ij}\left[\left(\cfrac{\sigma_{ij}}{r_{ij}}\right)^{12} - \left(\cfrac{\sigma_{ij}}{r_{ij}}\right)^{6}\right], \qquad r_{ij} > r_{ij}^{*}
    \end{cases}
$$

Alternatively,

$$
E_{VDW} = \cfrac{ACOEFF}{r^{12}_{ij}} - \cfrac{BCOEFF}{r^{6}_{ij}}
$$

For this we can compute first-hand the $ACOEFF$ and $BCOEFF$, and rewrite the equation such that:

$$
ACOEFF = \varepsilon_{ij} r^{12}_{0} = 4 \varepsilon_{ij} \sigma_{ij}^{12} \\
BCOEFF = 2\varepsilon_{ij} r^{6}_{0} = 4 \varepsilon_{ij} \sigma_{ij}^{6} \\
\varepsilon_{ij} = \sqrt{\varepsilon_{i}\varepsilon_{j}} \\
\sigma_{ij} = \cfrac{\sigma_{i} + \sigma_{j}}{2} \\
r_{0} = \sqrt[6]{2} \cdot \sigma_{ij} \\
r^{*}_{ij} = \alpha (r_{i} + r_{j}), \quad \alpha = 0.8
$$

Scale factors for atoms 3 bonds apart:

`lj14scale` * $ACOEFF$ ($BCOEFF$)


Units:

> sigma is in nm, and epsilon is in kJ/mole.

Importante readings:

* http://docs.openmm.org/latest/userguide/application.html#nonbondedforce

In [7]:
epsilons_l = []
for atom in gs.data_array:
    
    if atom[col_resSeq] == res_num[-1]:
        res = 'C' + atom[col_resName]
    
    else:
        res = atom[col_resName]
    
    aname = atom[col_name]
    
    atype = ff14SB[res][aname]['type']
    
    ep = float(ff14SB[atype]['epsilon'])
    epsilons_l.append(ep)


epsilons = np.array(epsilons_l)
print(epsilons[:10])
print(epsilons[-10:])
# confirmed first and last atom, seems okay 2020/11/27

[0.71128   0.4577296 0.359824  0.87864   0.4577296 0.0656888 0.0656888
 0.0656888 0.0656888 0.71128  ]
[0.4577296 0.359824  0.87864   0.4577296 0.0656888 0.0656888 0.0656888
 0.0656888 0.0656888 0.87864  ]


In [8]:
# these values have direct relationship with what reported in:
# Chernyshov, I. Yu., Ananyev, I. V. & Pidko, E. A. Revisiting van der Waals Radii: From Comprehensive Structural Analysis to Knowledge‐Based Classification of Interatomic Contacts. Chemphyschem 21, 370–376 (2020).
sigma_l = []
for atom in gs.data_array:
    
    if atom[col_resSeq] == res_num[-1]:
        res = 'C' + atom[col_resName]
    
    else:
        res = atom[col_resName]
    
    aname = atom[col_name]
    
    atype = ff14SB[res][aname]['type']
    
    sig = float(ff14SB[atype]['sigma']) * 10  # conversion to Angstroms
    sigma_l.append(sig)


sigmas = np.array(sigma_l)
print(sigmas[:10])
print(sigmas[-10:])
# confirmed first and last atom, seems okay 2020/11/27

[3.24999852 3.39966951 3.39966951 2.9599219  3.39966951 2.47135304
 2.64953279 2.64953279 2.64953279 3.24999852]
[3.39966951 3.39966951 2.9599219  3.39966951 1.06907846 2.47135304
 2.64953279 2.64953279 2.64953279 2.9599219 ]


#### Precomputing ACOEFF and BCOEFF

$\sigma$ and $\varepsilon$ values are as extracted from the `protein.ff14SB.xml`.

In [9]:
def calc_e_ij(epsilons):
    """Calculate epsilon for ij pair."""
    indices = np.triu_indices(epsilons.size, k=+1)    
    
    result = np.sqrt(np.outer(epsilons, epsilons)[indices])
    
    assert result.size == (epsilons.size * epsilons.size - epsilons.size) // 2
    return result

In [10]:
epsilons_ij = calc_e_ij(epsilons)
assert epsilons_ij[0] - np.sqrt(epsilons[0] * epsilons[0]) < 0.0000001

In [11]:
def calc_sig_ij(sigmas):
    """Calculates the sig_ij for all atom pairs."""
    indices = np.triu_indices(sigmas.size, k=+1)
    
    result = (sigmas[:, None] + sigmas)[indices] / 2
    
    _expected_size = (sigmas.size * sigmas.size - sigmas.size) // 2
    assert result.size == _expected_size, (result.size, _expected_size)
    return result

In [12]:
sigmas_ij = calc_sig_ij(sigmas)

In [13]:
acoeff = 4 * epsilons_ij * np.power(sigmas_ij, 12)
bcoeff = 4 * epsilons_ij * np.power(sigmas_ij, 6)

In [14]:
res_topology = generate_residue_template_topology()

In [15]:
bonds3 = topology_3_bonds_apart(res_topology)

In [16]:
res_names_ij = []
res_num_ij = []
atom_names_ij = []

rnames = res_names_ij.append
rnums = res_num_ij.append
anames = atom_names_ij.append

for i in range(len(gs) - 1):
    for j in range(i + 1, len(gs)):
        rnames((gs.data_array[i, col_resName], gs.data_array[j, col_resName]))
        rnums((gs.data_array[i, col_resSeq], gs.data_array[j, col_resSeq]))
        anames((gs.data_array[i, col_name], gs.data_array[j, col_name]))

In [17]:
zipit = zip(res_names_ij, res_num_ij, atom_names_ij)
bonds3_mask = np.full(len(res_names_ij), False)

In [18]:
counter = 0
for (rn1, rn2), (n1, n2), (a1, a2) in zipit:
    if n1 == n2 and a2 in bonds3[rn1][a1]:
        bonds3_mask[counter] = True
    
    
    elif a1 in inter_res_exact_3_bonds \
            and a2 in inter_res_exact_3_bonds[a1] \
            and int(n1) + 1 == int(n2):
        bonds3_mask[counter] = True
    
    counter += 1
# looks like is working

In [25]:
bonds3_mask[115:118]

array([False, False,  True])

In [23]:
for i, t in enumerate(atom_names_ij):
    print(i, t)

0 ('N', 'CA')
1 ('N', 'C')
2 ('N', 'O')
3 ('N', 'CB')
4 ('N', 'HA')
5 ('N', 'HB1')
6 ('N', 'HB2')
7 ('N', 'HB3')
8 ('N', 'N')
9 ('N', 'CA')
10 ('N', 'C')
11 ('N', 'O')
12 ('N', 'CB')
13 ('N', 'H')
14 ('N', 'HA')
15 ('N', 'HB1')
16 ('N', 'HB2')
17 ('N', 'HB3')
18 ('N', 'N')
19 ('N', 'CA')
20 ('N', 'C')
21 ('N', 'O')
22 ('N', 'CB')
23 ('N', 'H')
24 ('N', 'HA')
25 ('N', 'HB1')
26 ('N', 'HB2')
27 ('N', 'HB3')
28 ('N', 'N')
29 ('N', 'CA')
30 ('N', 'C')
31 ('N', 'O')
32 ('N', 'CB')
33 ('N', 'H')
34 ('N', 'HA')
35 ('N', 'HB1')
36 ('N', 'HB2')
37 ('N', 'HB3')
38 ('N', 'N')
39 ('N', 'CA')
40 ('N', 'C')
41 ('N', 'O')
42 ('N', 'CB')
43 ('N', 'H')
44 ('N', 'HA')
45 ('N', 'HB1')
46 ('N', 'HB2')
47 ('N', 'HB3')
48 ('N', 'N')
49 ('N', 'CA')
50 ('N', 'C')
51 ('N', 'O')
52 ('N', 'CB')
53 ('N', 'H')
54 ('N', 'HA')
55 ('N', 'HB1')
56 ('N', 'HB2')
57 ('N', 'HB3')
58 ('N', 'N')
59 ('N', 'CA')
60 ('N', 'C')
61 ('N', 'O')
62 ('N', 'CB')
63 ('N', 'H')
64 ('N', 'HA')
65 ('N', 'HB1')
66 ('N', 'HB2')
67 ('N', 'H

1447 ('H', 'HB2')
1448 ('H', 'HB3')
1449 ('H', 'N')
1450 ('H', 'CA')
1451 ('H', 'C')
1452 ('H', 'O')
1453 ('H', 'CB')
1454 ('H', 'H')
1455 ('H', 'HA')
1456 ('H', 'HB1')
1457 ('H', 'HB2')
1458 ('H', 'HB3')
1459 ('H', 'N')
1460 ('H', 'CA')
1461 ('H', 'C')
1462 ('H', 'O')
1463 ('H', 'CB')
1464 ('H', 'H')
1465 ('H', 'HA')
1466 ('H', 'HB1')
1467 ('H', 'HB2')
1468 ('H', 'HB3')
1469 ('H', 'N')
1470 ('H', 'CA')
1471 ('H', 'C')
1472 ('H', 'O')
1473 ('H', 'CB')
1474 ('H', 'H')
1475 ('H', 'HA')
1476 ('H', 'HB1')
1477 ('H', 'HB2')
1478 ('H', 'HB3')
1479 ('H', 'N')
1480 ('H', 'CA')
1481 ('H', 'C')
1482 ('H', 'O')
1483 ('H', 'CB')
1484 ('H', 'H')
1485 ('H', 'HA')
1486 ('H', 'HB1')
1487 ('H', 'HB2')
1488 ('H', 'HB3')
1489 ('H', 'N')
1490 ('H', 'CA')
1491 ('H', 'C')
1492 ('H', 'O')
1493 ('H', 'CB')
1494 ('H', 'H')
1495 ('H', 'HA')
1496 ('H', 'HB1')
1497 ('H', 'HB2')
1498 ('H', 'HB3')
1499 ('H', 'N')
1500 ('H', 'CA')
1501 ('H', 'C')
1502 ('H', 'O')
1503 ('H', 'CB')
1504 ('H', 'H')
1505 ('H', 'HA')
1506

2931 ('C', 'N')
2932 ('C', 'CA')
2933 ('C', 'C')
2934 ('C', 'O')
2935 ('C', 'CB')
2936 ('C', 'H')
2937 ('C', 'HA')
2938 ('C', 'HB1')
2939 ('C', 'HB2')
2940 ('C', 'HB3')
2941 ('C', 'N')
2942 ('C', 'CA')
2943 ('C', 'C')
2944 ('C', 'O')
2945 ('C', 'CB')
2946 ('C', 'H')
2947 ('C', 'HA')
2948 ('C', 'HB1')
2949 ('C', 'HB2')
2950 ('C', 'HB3')
2951 ('C', 'N')
2952 ('C', 'CA')
2953 ('C', 'C')
2954 ('C', 'O')
2955 ('C', 'CB')
2956 ('C', 'H')
2957 ('C', 'HA')
2958 ('C', 'HB1')
2959 ('C', 'HB2')
2960 ('C', 'HB3')
2961 ('C', 'N')
2962 ('C', 'CA')
2963 ('C', 'C')
2964 ('C', 'O')
2965 ('C', 'CB')
2966 ('C', 'H')
2967 ('C', 'HA')
2968 ('C', 'HB1')
2969 ('C', 'HB2')
2970 ('C', 'HB3')
2971 ('C', 'N')
2972 ('C', 'CA')
2973 ('C', 'C')
2974 ('C', 'O')
2975 ('C', 'CB')
2976 ('C', 'H')
2977 ('C', 'HA')
2978 ('C', 'HB1')
2979 ('C', 'HB2')
2980 ('C', 'HB3')
2981 ('C', 'N')
2982 ('C', 'CA')
2983 ('C', 'C')
2984 ('C', 'O')
2985 ('C', 'CB')
2986 ('C', 'H')
2987 ('C', 'HA')
2988 ('C', 'HB1')
2989 ('C', 'HB2')
2990

4514 ('HA', 'CA')
4515 ('HA', 'C')
4516 ('HA', 'O')
4517 ('HA', 'CB')
4518 ('HA', 'H')
4519 ('HA', 'HA')
4520 ('HA', 'HB1')
4521 ('HA', 'HB2')
4522 ('HA', 'HB3')
4523 ('HA', 'N')
4524 ('HA', 'CA')
4525 ('HA', 'C')
4526 ('HA', 'O')
4527 ('HA', 'CB')
4528 ('HA', 'H')
4529 ('HA', 'HA')
4530 ('HA', 'HB1')
4531 ('HA', 'HB2')
4532 ('HA', 'HB3')
4533 ('HA', 'N')
4534 ('HA', 'CA')
4535 ('HA', 'C')
4536 ('HA', 'O')
4537 ('HA', 'CB')
4538 ('HA', 'H')
4539 ('HA', 'HA')
4540 ('HA', 'HB1')
4541 ('HA', 'HB2')
4542 ('HA', 'HB3')
4543 ('HA', 'N')
4544 ('HA', 'CA')
4545 ('HA', 'C')
4546 ('HA', 'O')
4547 ('HA', 'CB')
4548 ('HA', 'H')
4549 ('HA', 'HA')
4550 ('HA', 'HB1')
4551 ('HA', 'HB2')
4552 ('HA', 'HB3')
4553 ('HA', 'N')
4554 ('HA', 'CA')
4555 ('HA', 'C')
4556 ('HA', 'O')
4557 ('HA', 'CB')
4558 ('HA', 'H')
4559 ('HA', 'HA')
4560 ('HA', 'HB1')
4561 ('HA', 'HB2')
4562 ('HA', 'HB3')
4563 ('HA', 'OXT')
4564 ('HB1', 'HB2')
4565 ('HB1', 'HB3')
4566 ('HB1', 'N')
4567 ('HB1', 'CA')
4568 ('HB1', 'C')
4569 ('H

In [ ]:
def calc_r0(rmin):
    """Calculates the r0 for all atom pairs."""
    indices = np.triu_indices(rmin.size, k=+1)
    result = (rmin[:, None] + rmin)[indices]
    _expected_size = (rmin.size * rmin.size - rmin.size) // 2
    assert result.size == _expected_size, (result.size, _expected_size)
    return result

In [ ]:
r0 = calc_r0(r_min)
print(r0[:5])

## Effective Born Radius

$$
R^{-1}_{i} (\mathring{A}) = \widetilde{\rho}^{-1}_{i} - \rho^{-1}_{i} \tanh(\alpha\Psi - \beta\Psi^{2} + \gamma\Psi^{3})
$$

Where,

$$
\rho_{i} (\text{atom radii in } \mathring{A}) \\
\widetilde{\rho}_{i} = \rho_{i} - 0.09 \\
\Psi = I\rho_{i} \\
I = scaling factor \\
\begin{vmatrix}
	\alpha \\
	\beta \\
	\gamma 
\end{vmatrix}\begin{vmatrix}
	0.8 & 1 \\
	0.0 & 0.8 \\
	2.91 & 4.85
\end{vmatrix}
$$



In [ ]:
def calc_Ri(pi, a, b, g, I, ptilde=0.09):
    """Calculates Ri according to Onufriev 2004."""
    ppi = pi - ptilde
    Y = I*ppi
    return (ppi**-1 - pi**-1 * np.tanh(a*Y - b*Y**2 + g*Y**3))**(-1)

### Demonstration of Ri

Bellow we reproduce Fig. 1 from Onufriev *et. al.* 2004.

In [ ]:
pi = 1.7  # for carbon atom

# generate a series of scaling I
I = np.linspace(0, 1, num=50)

# using first set of scalars
Ri_distribution_1 = calc_Ri(pi, 0.8, 0.0, 2.91, I)

# using second set of scalars
Ri_distribution_2 = calc_Ri(pi, 1, 0.8, 4.85, I)

In [ ]:
# plot results
plt.plot(I, Ri_distribution_1, color='black')
plt.plot(I, Ri_distribution_2, color='red')

# General Python Tests

In [ ]:
a  = np.char.array(list('abcd'))
print(a[:,None]+a)

In [ ]:
a

In [ ]:
a[:,None]+a

In [ ]:
b = np.arange(9)
b
il2 = np.triu_indices(9, k=1)

In [ ]:
(b[:, None]+b)[il2]

In [ ]:
il1 = np.triu_indices(4, k=+1)

In [ ]:
a = np.arange(16).reshape(4, 4)
a

In [ ]:
a[il1]

# Testing some values of the force field

In [ ]:
pi = 1.7
ppi = pi - 0.09
a = 0.8
b = 0
g = 2.91

I = np.linspace(0, 1, num=50)
Y = I*ppi

eq = (ppi**-1 - pi**-1 * np.tanh(a*Y - b*Y**2 + g*Y**3))**(-1)
eq2 = (ppi**-1 - pi**-1 * np.tanh(1*Y - 0.8*Y**2 + 4.85*Y**3))**(-1)

In [ ]:
import sympy

In [ ]:
rr = sympy.Symbol('r')
rho = sympy.Symbol('rho')

In [ ]:
sympy.integrate((rr-rho)*(1/rr**4), rr)

In [ ]:
help(str)